# Packages

In [23]:
using Pkg
#haskey(Pkg.installed(), "Languages") || Pkg.add("Languages")
using Knet, Plots, Statistics, LinearAlgebra, Random
using Languages

# Constants

In [24]:
#=Citation: Publications Using the Dataset. Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).=#
DATA_URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
FILE_NAME = "aclImdb"
DATA_DIR = "./Data"
NGRAM = 5
nothing

# Download and preprocess data

In [25]:
#define a struct for a review that has a text (list of words)and a label
struct review txt; label;end

#download review from DARA_URL
function _download_reviews()
    if !isdir(DATA_DIR)
        mkdir(DATA_DIR)
    end
    url = DATA_URL
    path = "$DATA_DIR/$FILE_NAME"
    if !isdir(path)
        println("Downloading the data from the internet...")
        file = "aclImdb_v1.tar"
        #file = download(url, path)
        println("Extracting the data...")
        run(`tar -xvf $(joinpath(DATA_DIR, file)) -C ./DATA_DIR/ -C`)
        println("Finished.")
        rm(file)
    end
    return path
end

#creat a dictionary of stopwords
_stopwords = Dict(x => true for x in stopwords(Languages.English()))

#clean a review by 1-removing punctionations and extra spaces, split into an array or words 
#removing stopwords, and covert ot lowercase. Then retern a list of words
function clean_text(txt)
    txt = replace(txt, r"[^a-zA-Z\s-]" => " ")
    txt = replace(txt, r"--" => " ")
    txt = replace(txt, r"\s+" => " ")
    lst = split(txt)
    lst = map(lowercase, lst)
    #&& get.(_stopwords) != true 
    lst = lst[[(length(i) > 2 && get(_stopwords, i, false) != true) for i in lst]]
    return lst
end

#extract review from the files insided a specific directory, clean the data and return a list of reviews struct
function _extract_reviews(path, label = 0)
    content = []
    for f in readdir(path)
        open(string(path,"/",f)) do re
            txt = read(re,String)
            #println(txt)
            push!(content, review(clean_text(txt), label))
        end
    end
    return content
end


_extract_reviews (generic function with 2 methods)

# Define an ngram model

In [26]:
function ngram(lst, n)
    new_lst = copy(lst)
    for i in 2:min(n,length(lst))
        for j in i:length(lst)
            push!(new_lst, join(lst[j-i+1:j], "_"))
        end
    end
    return new_lst
end
#proccess the data

ngram (generic function with 1 method)

# Training

In [27]:
#define Naive Bayes model
struct NaiveBayes2 class_prob; map_prob; end

#map words into numbers and calcualte the frequency
function map_word(dtrn)
    w2n = Dict()
    n2w = Dict()
    t = 1
    words_cnt = Dict()
    for re in dtrn
        lst = re.txt
        lbl = re.label
        for word in lst
            if !haskey(words_cnt, word)
                get!(w2n, word, t)
                get!(n2w, t, word)
                get!(words_cnt, word, fill(0, length(c2i)))
            end
            words_cnt[word][lbl] += 1
        end
    end
    words_cnt, w2n ,n2w
end
nothing

# Predict

In [32]:
#define predict function
function predict(txt, nb, eps = 10)
    prob = [log(nb.class_prob[c]) + sum([log(haskey(nb.map_prob, w) ? nb.map_prob[w][c]+eps : eps) for w in txt]) for c in 1:length(c2i)]
    ind = argmax(prob)
    #println("Model prediction is $(i2c[ind])")
    return ind
end
predict(txt::String, nb) = predict(txt)

#define accuracy function
function accuracy(dtst, nb)
    return mean([re.label == predict(re.txt, nb) for re in dtst])
end
nothing

# Benchmarks for different NGRAMS models

In [29]:
println("Downloading data..")
path = _download_reviews()
println("Extracting data..")
c2i = Dict("Positive" => 1, "Negative" => 2)
i2c = Dict(1 => "Positive", 2 => "Negative")
dtrn = [_extract_reviews(joinpath(path, "train", "pos"), 1); _extract_reviews(joinpath(path, "train", "neg"), 2)]
dtst = [_extract_reviews(joinpath(path, "test", "pos"), 1); _extract_reviews(joinpath(path, "test", "neg"), 2)]
println("Finsied extracting and cleaning $(length(dtrn)) training reviews and $(length(dtrn)) testing reviews")

Extracting data..
Finsied extracting and cleaning 25000 training reviews and 25000 testing reviews


In [33]:
for i in 1:6
    println("modeling data in $i gram model")
    dtrn2 = [review(ngram(re.txt, i),re.label) for re in dtrn]
    dtst2 = [review(ngram(re.txt, i),re.label) for re in dtst]
    words_cnt, w2n ,n2w = map_word(dtrn2)
    #find class probability
    c_p = [sum([re.label == c for re in dtrn]) for c in 1:length(c2i)]
    println("Naive Bayes model with $i gram model prodcued accuracy of $(accuracy(dtst2, NaiveBayes2(c_p, words_cnt)))")
end

modeling data in 1 gram model
Naive Bayes model with 1 gram model prodcued accuracy of 0.8342
modeling data in 2 gram model
Naive Bayes model with 2 gram model prodcued accuracy of 0.84492
modeling data in 3 gram model
Naive Bayes model with 3 gram model prodcued accuracy of 0.84588
modeling data in 4 gram model
Naive Bayes model with 4 gram model prodcued accuracy of 0.84584
modeling data in 5 gram model
Naive Bayes model with 5 gram model prodcued accuracy of 0.8458
modeling data in 6 gram model
Naive Bayes model with 6 gram model prodcued accuracy of 0.8458
